# Plotting SWOT outputs (from simulation in NATL60) 

## Imports

In [1]:
# no warnings in outputs
import warnings
warnings.filterwarnings('ignore')

# import libraries
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr # xarray missing !!!!
import cartopy.crs as ccrs # cartopy missing !!!!
import cartopy.feature as cfeature 

# imports to formate grid label
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import matplotlib.ticker as mticker

# import beautiful colormaps for o
import cmocean # cmocean missing !!!!

# import for spectral analysis 
from scipy import signal



%matplotlib inline
 

## Load files

In [10]:
# --- Read SSH fields --- #
# set path and directories
pathdata = '/Users/sammymetref/Documents/Boost-Swot/Notebooks/GitHub/Personal_Files/2018/Data/'
 

# set area to region1
r1lon1=-85
r1lon2=-35
r1lat1=30
r1lat2=50   
# set area to region1
r2lon1=-85
r2lon2=-35
r2lat1=30
r2lat2=50     

nmonths=16   
timesh1=[]
timesh2=[]
ilabel=[]
ilabelname=[]
itime=0
for imonths in range(nmonths):
    # filename
    if imonths+6<13:
        filename1 = 'NATL60-CJM165_y2012m'+np.str('{:02d}'.format(imonths+6))+'.1d.SSH.nc' # SSH field
    else: 
        filename1 = 'NATL60-CJM165_y2013m'+np.str('{:02d}'.format(imonths+6-12))+'.1d.SSH.nc' # SSH field
    
    ''.join([pathdata, filename1])

    # read file with xarray
    ds1 = xr.open_dataset(''.join([pathdata, filename1])) 

    # load variables
    lon1 = ds1['nav_lon'][:,:] 
    lat1 = ds1['nav_lat'][:,:] 
    
    # set indices area to region1  
    r1lon=lon1[(r1lon1<lon1[:,0]),]
    print(r1lon)
    # set indices area to region2
    ir2=(np.where((r2lon1<lon1[:,:]) & (lon1[:,:]<r2lon2) & (r2lat1<lat1[:,:]) & (lat1[:,:]<r2lat2)))[0]
    
    ndays=np.shape(ds1['sossheig'])[0] 
    for idays in range(0,ndays): 
        ssh1 = ds1['sossheig'][idays,ir1] 
        ssh2 = ds1['sossheig'][idays,ir2] 
        timesh1=np.append(timesh1,np.mean(np.mean(ssh1)))  
        timesh2=np.append(timesh2,np.mean(np.mean(ssh2))) 
        if idays==1 and imonths==0 or idays==15 and imonths!=0 :
            ilabel=np.append(ilabel,itime)
            if imonths+6<13:
                ilabelname=np.append(ilabelname,np.str('{:02d}'.format(imonths+6))+'/12')
            else: 
                ilabelname=np.append(ilabelname,np.str('{:02d}'.format(imonths+6-12))+'/13')
        itime=itime+1
         

IndexError: 2-dimensional boolean indexing is not supported. 

## Spectral plot for region1 and region2

In [ ]:
Fs = 1.0 # sampling rate: 1 every 24h
Ts = 1.0/Fs; # sampling interval
t = np.arange(0,np.shape(timesh1)[0]*Ts,Ts) # time vector

# Region 1
y1 = timesh1
n1 = len(y1) # length of the signal 
k1 = np.arange(n1)
T1 = n1/Fs 
frq1 = k1/T1 # two sides frequency range
frq1 = frq1[range(int(n/2))] # one side frequency range
wvnb1=2*np.pi*frq1
Y1 = np.fft.fft(y1)/n1 # fft computing and normalization
Y1 = Y1[range(int(n1/2))]

# Region 2
y2 = timesh2
n2 = len(y2) # length of the signal 
k2 = np.arange(n2)
T2 = n2/Fs 
frq2 = k2/T2 # two sides frequency range
frq2 = frq2[range(int(n2/2))] # one side frequency range
wvnb2=2*np.pi*frq2
Y2 = np.fft.fft(y2)/n2 # fft computing and normalization
Y2 = Y2[range(int(n2/2))]
 
# Plot    
fig = plt.figure(figsize=(25,12))
ax = fig.subplots(2, 2)
plt.suptitle('SSH')
# region1
ax[0,0].set_title('Region 1')
ax[0,0].plot(t,y1)
ax[0,0].set_xlabel('Time (days)')
ax[0,0].set_ylabel('Amplitude (m)')  
ax[1,0].plot(wvnb1,abs(Y1),'k') # plotting the spectrum
ax[1,0].set_yscale('log')
ax[1,0].set_xscale('log')
ax[1,0].set_xlabel('wavenumber');
ax[1,0].set_ylabel('Spectrum (m/cpd)');
ax[1,0].axis('tight');
# region2
ax[0,1].set_title('Region 2')
ax[0,1].plot(t,y2)
ax[0,1].set_xlabel('Time (days)')
ax[0,1].set_ylabel('Amplitude (m)')  
ax[1,1].plot(wvnb2,abs(Y2),'k') # plotting the spectrum
ax[1,1].set_yscale('log')
ax[1,1].set_xscale('log')
ax[1,1].set_xlabel('wavenumber');
ax[1,1].set_ylabel('Spectrum (m/cpd)');
ax[1,1].axis('tight');
  